In [28]:
from bs4 import BeautifulSoup
import pandas as pd

In [29]:
def extract_entries_from_file(file_path):
    entries = []
    with open(file_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')
        for entry in soup.find_all('a', attrs={"name": True}):
            try:
                alabama_word = entry['name']
            except:
                continue
            try:
                definition = entry.find_next()
                if (definition.get_text().strip() == "Var. of"):
                    definition = definition.get_text().strip() + " " + definition.find_next().get_text().strip()
                elif (entry.find_next().find_previous(string=True).strip() == "Neg." or entry.find_next().find_previous(string=True).strip() == "Neg.\n ☞" or 'Neg.' in entry.find_next().find_previous(string=True).strip()):
                    definition = "Negative form of " + entry.find_next('a').get_text().strip()
                elif (definition.get_text().strip() == "1" or definition.get_text().strip() == "2"):
                    definition = definition.find_next().get_text().strip()
                elif (definition.get_text().strip()[0] == '(' and file_path == '/Users/jacobfernandes/Desktop/alabamadictionary/DictfilesAlabama/Affixes.txt'):
                    definition = definition.get_text().strip() + " " + definition.find_next().get_text().strip()
                else:
                    definition = definition.get_text().strip()
            except:
                print("HELP")
            l_class = ""
            if (entry.find_next().find_next().name == "font"):
                text = entry.find_next().find_next()
                if (text.get('color') == '#800080'):
                    l_class = text.get_text().strip()
                elif (len(definition) >= 3):
                    try:
                        if (definition[0:3] == "to "):
                            l_class = "Other Verb"
                    except:
                        ()
            entries.append({
                'Alabama': alabama_word,
                'English': definition,
                'Class' : l_class,
            })
    return entries

In [30]:
extract_entries_from_file('/Users/jacobfernandes/Desktop/alabamadictionary/DictfilesAlabama/Affixes.txt')

[{'Alabama': '-a',
  'English': '(Nom/Mid) (nonproductive) object nominalizing suffix or middle',
  'Class': ''},
 {'Alabama': 'a-',
  'English': '(Appl) (nonproductive) verb stem derivational prefix typically adding an object',
  'Class': ''},
 {'Alabama': '-aa', 'English': '(Pron) I', 'Class': ''},
 {'Alabama': 'aa-', 'English': '(GenLoc) at, to', 'Class': ''},
 {'Alabama': '-aaba',
  'English': '(Modl) verb suffix indicating an opinion or belief or that the speaker is asking for confirmation of a belief, as in tag questions',
  'Class': ''},
 {'Alabama': '-áahosi', 'English': '(Adv) almost', 'Class': ''},
 {'Alabama': '-áali',
  'English': "(Modf) the one, the very, the same, the exact; one's very own, one's own self",
  'Class': ''},
 {'Alabama': '-àapi', 'English': '(Adv) really, very; but', 'Class': ''},
 {'Alabama': 'aati-',
  'English': '(PlObj) verb prefix typically indicating a plural third person object, often unspecified, or occasionally a third person unspecified subject',

In [31]:
import os

# Assuming the extract_entries_from_file function is defined as described earlier

folder_path = "DictfilesAlabama"

# Get list of files in the folder
file_names = os.listdir(folder_path)

# Filter only .txt files
txt_files = [file for file in file_names if file.endswith('.txt')]
entries = []
# Iterate over each file and apply the extract_entries_from_file function
for file_name in txt_files:
    file_path = os.path.join(folder_path, file_name)
    entries += extract_entries_from_file(file_path)
    # Do something with the extracted entries, such as saving to a DataFrame
    # For example:
    # df = pd.DataFrame(entries)
    # print(df)


HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP
HELP


In [32]:
import pandas as pd

In [33]:
entries = pd.DataFrame(entries)
entries

,Alabama,English,Class
0,kàano,"to be well, all right, feel better",CHA-
1,káanon naaɬiíkko,to be unable to talk plainly (as after a strok...,
2,káasok,"but, though, however",
3,kabòska,caboose,
4,kachàali,"to bite repeatedly, chew",-LI/CHA-
...,...,...,...
8808,intonotli istiⁿhisli,wheel spokes,
8809,intoobi,Var. of intohbi,
8810,intòono,Var. of intòhno,
8811,intosoyli,to pour for,-LI/AM-/3


In [38]:
entries.loc[entries['Alabama'] == '-ba']

,Alabama,English,Class
6545,-ba,(Imp),


In [35]:
entries.to_csv('dictionary.csv')

In [36]:
with open('dictionary.csv', 'r') as f:
    with open('dict.json', 'w') as g:
        g.write("""{ "words" : [""")
        x = pd.read_csv(f)
        for index, row in x.iterrows():
            g.write("""
    { 
        "lemma":\"""" +  str(row['Alabama']) + """\",
        "definition": \"""" + str(row['English']) + """\",
        "class": \"""" + str(row['Class']) + """\"
    },""")
        g.write("]}")